### Rich-resource setting: Fine-tuning data sizes are [500, 600, 700, 800, 900, 1000, 2000]

In [1]:
import pandas as pd
import os, re, copy
from glob import glob
from reports import get_average_classification_report
pd.options.display.float_format = '{:.4f}'.format
from functools import reduce
pd.options.display.float_format = '{:.4f}'.format
import seaborn as sns

In [2]:
target_trained_with_nums = [500, 600, 700, 800, 900, 1000, 2000]
dirnames = sorted(glob(os.path.join('C:\DATA\ComBERT', 'results_spamFiltering', '*_seed4')))
len(dirnames)

32

In [4]:
dfs = []
for dirname in dirnames:
    test_company, model_name, method_name, epoch_num, seed_num = os.path.basename(dirname).split('_')
    
    records = []
    for target_trained_with_num in target_trained_with_nums:
        filepaths = glob(os.path.join(os.path.dirname(dirname), '{}_*'.format('_'.join([test_company, model_name, method_name, epoch_num])), \
                  'classification_report_train_{}.csv'.format(target_trained_with_num)))
        
        column_name = '_'.join([test_company, model_name, method_name, epoch_num])
        averaged_classification_report_df = get_average_classification_report([pd.read_csv(filepath) for filepath in filepaths])
        acc = 100*averaged_classification_report_df.filter(items = ['accuracy'], axis=0)['f1-score'].values[0]
        records.append((target_trained_with_num, acc))
    dfs.append(pd.DataFrame(records, columns=['trained_with', column_name]).sort_values(by=['trained_with']))

result = reduce(lambda df1,df2: pd.merge(df1,df2,on='trained_with'), dfs)
result.set_index('trained_with', inplace=True)
result.loc['Average'] = result.mean()

result

,testCompanyName=n_Araci_CM_epoch7,testCompanyName=n_Araci_NoPT_epoch7,testCompanyName=n_Araci_SM_epoch7,testCompanyName=n_Araci_WWM_epoch7,testCompanyName=n_BERT_CM_epoch7,testCompanyName=n_BERT_NoPT_epoch7,testCompanyName=n_BERT_SM_epoch7,testCompanyName=n_BERT_WWM_epoch7,testCompanyName=n_SECBERT_CM_epoch7,testCompanyName=n_SECBERT_NoPT_epoch7,...,testCompanyName=y_BERT_SM_epoch7,testCompanyName=y_BERT_WWM_epoch7,testCompanyName=y_SECBERT_CM_epoch7,testCompanyName=y_SECBERT_NoPT_epoch7,testCompanyName=y_SECBERT_SM_epoch7,testCompanyName=y_SECBERT_WWM_epoch7,testCompanyName=y_Yang_CM_epoch7,testCompanyName=y_Yang_NoPT_epoch7,testCompanyName=y_Yang_SM_epoch7,testCompanyName=y_Yang_WWM_epoch7
trained_with,,,,,,,,,,,,,,,,,,,,,
500,79.4000,74.0200,79.7400,78.0600,74.2800,79.3200,76.6800,76.1200,79.2000,78.1400,...,79.2600,79.7000,80.2600,77.5600,78.9400,80.5400,78.9800,78.8800,79.8800,79.7000
600,79.9800,73.8000,79.8000,79.6200,77.8400,79.5800,79.5000,78.3000,80.8000,77.9200,...,80.2400,77.6600,79.3600,77.6600,79.7200,80.1400,79.0400,78.8400,81.1400,80.5200
700,77.8800,76.6400,78.8200,78.5200,77.7200,77.1200,78.7400,77.2400,79.8400,76.9600,...,79.8200,80.1600,80.5400,79.6600,79.4800,80.3200,80.0200,80.0400,82.1800,80.9400
800,78.9800,74.1200,79.6200,77.6400,78.8800,76.5400,78.6000,76.2400,79.6400,77.4000,...,81.2800,80.9200,81.3200,80.0600,80.2400,79.7400,80.9800,80.4400,81.0400,81.0000
900,79.9600,76.7400,80.3200,78.0800,78.9800,77.0600,79.8400,78.6200,80.0800,77.6600,...,81.0000,81.3400,80.5000,80.2400,80.6600,81.1200,80.9400,81.0600,81.2200,81.7200
1000,79.4200,77.5400,79.4200,79.9200,79.1800,80.2600,80.0000,80.5000,79.2600,80.1000,...,81.5000,82.0000,81.9000,81.7800,79.9400,81.3800,81.4000,81.5800,82.0200,82.1000
2000,80.0400,79.5600,79.2800,80.5400,78.3000,81.6200,78.7800,79.3000,80.0000,81.3400,...,81.9000,81.9000,82.1000,81.3000,81.9000,82.3200,82.2200,81.2000,82.3200,83.1200
Average,79.3800,76.0600,79.5714,78.9114,77.8829,78.7857,78.8771,78.0457,79.8314,78.5029,...,80.7143,80.5257,80.8543,79.7514,80.1257,80.7943,80.5114,80.2914,81.4000,81.3000


### Evalautaion of improved performance of each model
* All models achieved high performance over 75.00.
* Based on the average scores in the test setting using tweets containing company names, **SEC-BERT** using the proposed company name masking method achieved *slightly high* spam filtering performance.
    - **yCM** > (yNoPT, ySM, yWWM)

* Based on the average scores in the test setting using tweets containing *no* company names, **SEC-BERT** using the proposed company name masking method achieved *slightly high* spam filtering performance.
    - **nCM** > (nNoPT, nSM, nWWM)

In [5]:
def show_table(model_name):
    print(model_name)
    dfs = []
    for setting in ['testCompanyName=n', 'testCompanyName=y']:
        one_model_setting_result = result[[col for col in result.columns if (model_name in col) and (setting in col)]]
        one_model_setting_result.columns = [item.replace(setting, setting[-1]).replace(model_name, '')\
                                            .replace('epoch7', '').replace('_', '')\
                                            for item in one_model_setting_result.columns]
        dfs.append(one_model_setting_result)

    one_model_result = reduce(lambda df1,df2: pd.merge(df1,df2,on='trained_with'), dfs)

    return one_model_result[sorted(one_model_result.columns)]

In [6]:
show_table('SECBERT')

SECBERT


,nCM,nNoPT,nSM,nWWM,yCM,yNoPT,ySM,yWWM
trained_with,,,,,,,,
500,79.2000,78.1400,76.6000,76.8400,80.2600,77.5600,78.9400,80.5400
600,80.8000,77.9200,78.2600,78.0200,79.3600,77.6600,79.7200,80.1400
700,79.8400,76.9600,78.2400,75.9000,80.5400,79.6600,79.4800,80.3200
800,79.6400,77.4000,77.4800,76.6600,81.3200,80.0600,80.2400,79.7400
900,80.0800,77.6600,76.2400,78.7200,80.5000,80.2400,80.6600,81.1200
1000,79.2600,80.1000,77.0600,78.9200,81.9000,81.7800,79.9400,81.3800
2000,80.0000,81.3400,80.1000,80.2400,82.1000,81.3000,81.9000,82.3200
Average,79.8314,78.5029,77.7114,77.9000,80.8543,79.7514,80.1257,80.7943


In [7]:
show_table('Yang')

Yang


,nCM,nNoPT,nSM,nWWM,yCM,yNoPT,ySM,yWWM
trained_with,,,,,,,,
500,77.9000,76.7800,79.3200,78.9600,78.9800,78.8800,79.8800,79.7000
600,79.0000,78.7400,79.7200,80.0600,79.0400,78.8400,81.1400,80.5200
700,78.4600,78.3200,79.2800,79.3600,80.0200,80.0400,82.1800,80.9400
800,78.5800,78.9600,79.2200,78.7600,80.9800,80.4400,81.0400,81.0000
900,79.5200,78.7000,80.0200,80.0600,80.9400,81.0600,81.2200,81.7200
1000,77.9800,79.3400,78.9600,80.1200,81.4000,81.5800,82.0200,82.1000
2000,77.1200,80.9400,81.7800,82.0200,82.2200,81.2000,82.3200,83.1200
Average,78.3657,78.8257,79.7571,79.9057,80.5114,80.2914,81.4000,81.3000


In [8]:
show_table('Araci')

Araci


,nCM,nNoPT,nSM,nWWM,yCM,yNoPT,ySM,yWWM
trained_with,,,,,,,,
500,79.4000,74.0200,79.7400,78.0600,79.9200,77.4600,79.8400,79.3800
600,79.9800,73.8000,79.8000,79.6200,80.4600,77.1800,80.8000,80.5800
700,77.8800,76.6400,78.8200,78.5200,79.9600,76.9800,80.0400,80.8000
800,78.9800,74.1200,79.6200,77.6400,80.5200,79.6200,80.6400,80.5200
900,79.9600,76.7400,80.3200,78.0800,80.4600,79.8000,79.9800,81.3800
1000,79.4200,77.5400,79.4200,79.9200,81.0000,80.6800,80.6400,81.6600
2000,80.0400,79.5600,79.2800,80.5400,81.1400,82.0000,81.6400,82.2800
Average,79.3800,76.0600,79.5714,78.9114,80.4943,79.1029,80.5114,80.9429


In [9]:
show_table('_BERT_')

_BERT_


,nCM,nNoPT,nSM,nWWM,yCM,yNoPT,ySM,yWWM
trained_with,,,,,,,,
500,74.2800,79.3200,76.6800,76.1200,78.9400,79.4400,79.2600,79.7000
600,77.8400,79.5800,79.5000,78.3000,79.5400,79.2400,80.2400,77.6600
700,77.7200,77.1200,78.7400,77.2400,80.5600,80.7400,79.8200,80.1600
800,78.8800,76.5400,78.6000,76.2400,80.9600,81.6400,81.2800,80.9200
900,78.9800,77.0600,79.8400,78.6200,80.8600,81.4400,81.0000,81.3400
1000,79.1800,80.2600,80.0000,80.5000,81.5600,81.8400,81.5000,82.0000
2000,78.3000,81.6200,78.7800,79.3000,81.8200,82.0600,81.9000,81.9000
Average,77.8829,78.7857,78.8771,78.0457,80.6057,80.9143,80.7143,80.5257
